In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import random
import lr_scheduler
import func
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler


folder_path = "F:/code/barlow/UR5"
file_pairs = func.load_file_pairs(folder_path)

AUTO = tf.data.AUTOTUNE
CROP_TO = 32
SEED = 200

PROJECT_DIM = 2048
BATCH_SIZE = 32
EPOCHS = 50

In [2]:
train_similar, train_dissimilar, test_similar, test_dissimilar = func.split_and_shuffle_pairs(file_pairs, folder_path)

In [3]:
def process_data(data_list):
    a = []
    b = []
    for jpg_path, npy_path in data_list:
        a.append(func.read_jpg_files(jpg_path))
        b.append(func.read_and_parse_npy_file(npy_path))
    return a,b

In [4]:
xtrain1 = process_data(train_similar)
xtrain2 = process_data(train_dissimilar)
xtest1 = process_data(test_similar)
xtest2 = process_data(test_dissimilar)

# label 1 - similar, 0 - disimilar
ytrain1 = [1] * 500
ytrain2 = [0] * 500
ytest1 = [1] *79
ytest2 = [0] *79

In [5]:
#
t1 = np.array([np.expand_dims(item, axis=-1) for item in xtrain1[0]])  # Expand dims
t1 = t1.astype("float32") / 255.0  # Normalize to [0, 1]
scaler = StandardScaler()
t2 = np.array([item for item in xtrain1[1]])  # NPY input
t2 = scaler.fit_transform(t2)  # Standardize to mean=0, std=1
train_ds1 = tf.data.Dataset.from_tensor_slices(((t1, t2), ytrain1))

#
t3 = np.array([np.expand_dims(item, axis=-1) for item in xtrain2[0]])  # Expand dims
t3 = t3.astype("float32") / 255.0  # Normalize to [0, 1]
t4 = np.array([item for item in xtrain2[1]])  # NPY input
t4 = scaler.transform(t4)  # Sử dụng scaler đã fit từ trước
train_ds2 = tf.data.Dataset.from_tensor_slices(((t3, t4), ytrain2))


# concat
train_ds = train_ds1.concatenate(train_ds2)
train_ds = train_ds.shuffle(buffer_size=1000, seed=226)
train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)


In [6]:
#
t5 = np.array([np.expand_dims(item, axis=-1) for item in xtest1[0]])  # Expand dims
t5 = t5.astype("float32") / 255.0  # Normalize to [0, 1]
t6 = np.array([item for item in xtest1[1]])  # NPY input
t6 = scaler.transform(t6)  # Sử dụng scaler đã fit từ dữ liệu training
test_ds1 = tf.data.Dataset.from_tensor_slices(((t5, t6), ytest1))

#
t7 = np.array([np.expand_dims(item, axis=-1) for item in xtest2[0]])  # Expand dims
t7 = t7.astype("float32") / 255.0  # Normalize to [0, 1]
t8 = np.array([item for item in xtest2[1]])  # NPY input
t8 = scaler.transform(t8)  # Sử dụng scaler đã fit từ dữ liệu training
test_ds2 = tf.data.Dataset.from_tensor_slices(((t7, t8), ytest2))

#
test_ds = test_ds1.concatenate(test_ds2)
test_ds = test_ds.shuffle(buffer_size=1000, seed=226)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

In [7]:
def concat_encoder():
    # Input đầu vào cho ảnh và npy_file
    image_input = layers.Input(shape=(128, 128, 1), name="image_input")  # Đầu vào ảnh 128x128x1
    npy_input = layers.Input(shape=(8,), name="npy_input")  # Đầu vào npy_file shape (8,)

    # x1: CNN đọc ảnh 128x128 và trả output shape (20)
    x1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(image_input)
    x1 = layers.MaxPooling2D((2, 2))(x1)
    x1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x1)
    x1 = layers.MaxPooling2D((2, 2))(x1)
    x1 = layers.Flatten()(x1)
    x1 = layers.Dense(20, activation='relu', name="cnn_output")(x1)

    # x2: Đọc npy_file với shape (8)
    # x2 = layers.Dense(8, activation='relu', name="npy_output")(npy_input)
    # x3: Kết hợp x1 + x2
    #x3 = layers.Concatenate(name="combined_features")([x1, x2])
    
    # Đọc thẳng npy_file với shape (8)
    x3 = layers.Concatenate(name="combined_features")([x1, npy_input])

    # Fully connected layers
    x = layers.Dense(128, activation="relu", name="dense_128")(x3)
    x = layers.Dense(64, activation="relu", name="dense_64")(x)
    x = layers.Dense(32, activation=None, name="projection_head")(x)  # Projection head

    # Model
    model = models.Model(inputs=[image_input, npy_input], outputs=x, name="encoder")
    return model

In [8]:
# Tạo encoder mới
encoder = concat_encoder()
barlow_twins = func.BarlowTwins(encoder=encoder)


backbone = tf.keras.Model(
    inputs=barlow_twins.encoder.input,
    outputs=barlow_twins.encoder.layers[-1].output  # output từ lớp Dense(32)
)
# Freeze 
barlow_twins.load_weights('bl_test2.weights.h5')
backbone.trainable = False

In [9]:
inputs = backbone.input
x = backbone.output
# Thêm lớp classifier phía sau
outputs = layers.Dense(2, activation="relu", name="classifier")(x)

# Xây dựng model mới
test_model = tf.keras.Model(inputs, outputs, name="test_model")

# Hiển thị kiến trúc model
test_model.summary()

Model: "test_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │        320 │ image_input[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 64, 64,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 65536)     │          0 │ max_pooling2d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_output (Dense)  │ (None, 20)        │  1,310,740 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ npy_input           │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ combined_features   │ (None, 28)        │          0 │ cnn_output[0][0], │
│ (Concatenate)       │                   │            │ npy_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_128 (Dense)   │ (None, 128)       │      3,712 │ combined_feature… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_64 (Dense)    │ (None, 64)        │      8,256 │ dense_128[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ projection_head     │ (None, 32)        │      2,080 │ dense_64[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ classifier (Dense)  │ (None, 2)         │         66 │ projection_head[… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,343,670 (5.13 MB)

 Trainable params: 66 (264.00 B)

 Non-trainable params: 1,343,604 (5.13 MB)

In [10]:
# Compile model
test_model.compile(
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.00005, momentum=0.9)
)

history = test_model.fit(
    train_ds, validation_data=test_ds, epochs=100
)

_, test_acc = test_model.evaluate(test_ds)
print("Test accuracy: {:.2f}%".format(test_acc * 100))

Epoch 1/100


f:\code\barlow\env\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['image_input', 'npy_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.4838 - loss: 3.4716 - val_accuracy: 0.4684 - val_loss: 2.0933
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5073 - loss: 2.1757 - val_accuracy: 0.4810 - val_loss: 1.7998
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.4920 - loss: 1.9502 - val_accuracy: 0.4684 - val_loss: 1.1167
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4888 - loss: 1.2574 - val_accuracy: 0.5000 - val_loss: 2.5569
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5106 - loss: 1.9760 - val_accuracy: 0.5000 - val_loss: 2.2495
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4886 - loss: 1.5548 - val_accuracy: 0.5000 - val_loss: 1.3157
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4928 - loss: 1.0672 - val_accuracy: 0.5000 - val_loss: 1.1437
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4953 - loss: 0.9993 - val_accuracy: 0.5000 - val_l

In [12]:
# Predict on the test dataset
predictions = test_model.predict(test_ds)

# Extract the true labels from the test dataset
true_labels = np.concatenate([y for x, y in test_ds], axis=0)

# Print predictions and true labels
for i, prediction in enumerate(predictions):
    predicted_label = np.argmax(prediction)  # Get the predicted class index
    print(f"Sample {i}: Predicted Label = {predicted_label}, True Label = {true_labels[i]}")

1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Sample 0: Predicted Label = 0, True Label = 1
Sample 1: Predicted Label = 1, True Label = 1
Sample 2: Predicted Label = 1, True Label = 0
Sample 3: Predicted Label = 0, True Label = 0
Sample 4: Predicted Label = 0, True Label = 0
Sample 5: Predicted Label = 0, True Label = 1
Sample 6: Predicted Label = 0, True Label = 1
Sample 7: Predicted Label = 1, True Label = 1
Sample 8: Predicted Label = 1, True Label = 1
Sample 9: Predicted Label = 0, True Label = 1
Sample 10: Predicted Label = 1, True Label = 0
Sample 11: Predicted Label = 1, True Label = 1
Sample 12: Predicted Label = 1, True Label = 0
Sample 13: Predicted Label = 1, True Label = 0
Sample 14: Predicted Label = 0, True Label = 1
Sample 15: Predicted Label = 0, True Label = 0
Sample 16: Predicted Label = 0, True Label = 0
Sample 17: Predicted Label = 0, True Label = 0
Sample 18: Predicted Label = 0, True Label = 1
Sample 19: Predicted Label = 0, True Label = 0
Sample 20: Predicted Label = 0, 